In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, Activation, Dropout, Add, LSTM, GRU, RNN, LayerNormalization, BatchNormalization, Conv1D, MaxPooling1D, Flatten, Layer
from keras import backend as K
from keras.optimizers import Adam,SGD
import tensorflow as tf
from keras import Model, regularizers, activations
from keras.constraints import Constraint
import pickle
from keras.utils import to_categorical, plot_model
from keras.datasets import mnist

In [6]:
def add_salt_and_pepper_noise(images, amount):
    noisy_images = images.copy()
    num_images = images.shape[0]
    num_pixels = images.shape[1] * images.shape[2]
    num_salt = np.ceil(amount * num_pixels)
    for i in range(num_images):
        # Add salt noise
        salt_indices = np.random.choice(num_pixels, size=int(num_salt), replace=False)
        noisy_images[i].flat[salt_indices] = 1
        
        # Add pepper noise
        pepper_indices = np.random.choice(num_pixels, size=int(num_salt), replace=False)
        noisy_images[i].flat[pepper_indices] = 0
    return noisy_images

In [7]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, SimpleRNN
from keras.utils import to_categorical, plot_model
from keras.datasets import mnist, cifar10

tf.random.set_seed(42)

training_acc = [0]*10
test_acc = [0]*10
test_loss = [0]*10
noise = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for i in range(10):  
    # load mnist dataset
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    # compute the number of labels
    num_labels = len(np.unique(y_train))

    # convert to one-hot vector
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    # resize and normalize
    image_size = X_train.shape[1]
    X_train = np.reshape(X_train,[-1, image_size, image_size])
    X_test = np.reshape(X_test,[-1, image_size, image_size])
    X_train = X_train.astype('float32') / 255
    X_test = X_test.astype('float32') / 255
    
    X_train = add_salt_and_pepper_noise(X_train, noise[i])

    input = Input(shape=(X_train.shape[1],X_train.shape[2]))
    x = LSTM(64, activation='relu', return_sequences=True)(input)
    x = LSTM(64, activation='relu', return_sequences=False)(x)
    x = Dense(num_labels, activation='softmax')(x)
    model = Model(input, x)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=100, batch_size=128, validation_split=0.25, verbose=2)

    training_acc[i] = history.history['val_accuracy'][-1]
    print(training_acc)
    
    loss, acc = model.evaluate(X_test, y_test, batch_size=128)

    test_acc[i] = acc
    print(test_acc)

    name = 'lstm_64_'
    name = name + str(noise[i]) + '.h5'
    model.save(name)

model.summary()

Epoch 1/100
352/352 - 51s - loss: 0.8687 - accuracy: 0.7122 - val_loss: 0.3246 - val_accuracy: 0.8998 - 51s/epoch - 144ms/step
Epoch 2/100
352/352 - 42s - loss: 0.2317 - accuracy: 0.9278 - val_loss: 0.1764 - val_accuracy: 0.9467 - 42s/epoch - 118ms/step
Epoch 3/100
352/352 - 42s - loss: 0.1534 - accuracy: 0.9528 - val_loss: 0.1407 - val_accuracy: 0.9569 - 42s/epoch - 120ms/step
Epoch 4/100
352/352 - 44s - loss: 0.1170 - accuracy: 0.9639 - val_loss: 0.1161 - val_accuracy: 0.9656 - 44s/epoch - 126ms/step
Epoch 5/100
352/352 - 46s - loss: 0.0928 - accuracy: 0.9705 - val_loss: 0.1094 - val_accuracy: 0.9681 - 46s/epoch - 132ms/step
Epoch 6/100
352/352 - 53s - loss: 0.0829 - accuracy: 0.9741 - val_loss: 0.1003 - val_accuracy: 0.9692 - 53s/epoch - 150ms/step
Epoch 7/100
352/352 - 56s - loss: 0.0761 - accuracy: 0.9763 - val_loss: 0.0856 - val_accuracy: 0.9744 - 56s/epoch - 160ms/step
Epoch 8/100
352/352 - 56s - loss: 0.0650 - accuracy: 0.9802 - val_loss: 0.0907 - val_accuracy: 0.9715 - 56s/epo

352/352 - 56s - loss: 0.0024 - accuracy: 0.9992 - val_loss: 0.0610 - val_accuracy: 0.9887 - 56s/epoch - 159ms/step
Epoch 66/100
352/352 - 54s - loss: 0.0054 - accuracy: 0.9983 - val_loss: 0.0645 - val_accuracy: 0.9872 - 54s/epoch - 154ms/step
Epoch 67/100
352/352 - 55s - loss: 0.0073 - accuracy: 0.9978 - val_loss: 0.0828 - val_accuracy: 0.9853 - 55s/epoch - 156ms/step
Epoch 68/100
352/352 - 49s - loss: 0.0044 - accuracy: 0.9986 - val_loss: 0.0599 - val_accuracy: 0.9889 - 49s/epoch - 140ms/step
Epoch 69/100
352/352 - 56s - loss: 0.0024 - accuracy: 0.9993 - val_loss: 0.0905 - val_accuracy: 0.9846 - 56s/epoch - 159ms/step
Epoch 70/100
352/352 - 54s - loss: 0.0064 - accuracy: 0.9978 - val_loss: 0.0717 - val_accuracy: 0.9869 - 54s/epoch - 152ms/step
Epoch 71/100
352/352 - 57s - loss: 0.0039 - accuracy: 0.9987 - val_loss: 0.0693 - val_accuracy: 0.9877 - 57s/epoch - 163ms/step
Epoch 72/100
352/352 - 58s - loss: 0.0021 - accuracy: 0.9994 - val_loss: 0.0740 - val_accuracy: 0.9876 - 58s/epoch - 

Epoch 28/100
352/352 - 49s - loss: 0.0290 - accuracy: 0.9905 - val_loss: 0.1060 - val_accuracy: 0.9715 - 49s/epoch - 138ms/step
Epoch 29/100
352/352 - 50s - loss: 0.0295 - accuracy: 0.9894 - val_loss: 0.1216 - val_accuracy: 0.9701 - 50s/epoch - 142ms/step
Epoch 30/100
352/352 - 50s - loss: 0.0257 - accuracy: 0.9910 - val_loss: 0.1244 - val_accuracy: 0.9702 - 50s/epoch - 143ms/step
Epoch 31/100
352/352 - 49s - loss: 0.0267 - accuracy: 0.9914 - val_loss: 0.1180 - val_accuracy: 0.9701 - 49s/epoch - 139ms/step
Epoch 32/100
352/352 - 48s - loss: 0.0217 - accuracy: 0.9925 - val_loss: 0.1181 - val_accuracy: 0.9699 - 48s/epoch - 136ms/step
Epoch 33/100
352/352 - 50s - loss: 0.0260 - accuracy: 0.9913 - val_loss: 0.1344 - val_accuracy: 0.9659 - 50s/epoch - 142ms/step
Epoch 34/100
352/352 - 50s - loss: 0.0191 - accuracy: 0.9933 - val_loss: 0.1246 - val_accuracy: 0.9691 - 50s/epoch - 143ms/step
Epoch 35/100
352/352 - 49s - loss: 0.0213 - accuracy: 0.9929 - val_loss: 0.1280 - val_accuracy: 0.9678 -

352/352 - 49s - loss: 0.0031 - accuracy: 0.9990 - val_loss: 0.1608 - val_accuracy: 0.9743 - 49s/epoch - 138ms/step
Epoch 93/100
352/352 - 47s - loss: 0.0089 - accuracy: 0.9976 - val_loss: 0.1359 - val_accuracy: 0.9737 - 47s/epoch - 133ms/step
Epoch 94/100
352/352 - 47s - loss: 0.0047 - accuracy: 0.9987 - val_loss: 0.1438 - val_accuracy: 0.9731 - 47s/epoch - 132ms/step
Epoch 95/100
352/352 - 44s - loss: 0.0067 - accuracy: 0.9978 - val_loss: 0.1413 - val_accuracy: 0.9741 - 44s/epoch - 125ms/step
Epoch 96/100
352/352 - 47s - loss: 0.0017 - accuracy: 0.9994 - val_loss: 0.1454 - val_accuracy: 0.9749 - 47s/epoch - 133ms/step
Epoch 97/100
352/352 - 47s - loss: 6.8564e-04 - accuracy: 0.9998 - val_loss: 0.1839 - val_accuracy: 0.9722 - 47s/epoch - 132ms/step
Epoch 98/100
352/352 - 47s - loss: 0.0107 - accuracy: 0.9967 - val_loss: 0.1267 - val_accuracy: 0.9744 - 47s/epoch - 134ms/step
Epoch 99/100
352/352 - 45s - loss: 0.0062 - accuracy: 0.9982 - val_loss: 0.1406 - val_accuracy: 0.9743 - 45s/epoc

Epoch 55/100
352/352 - 24s - loss: 0.0223 - accuracy: 0.9926 - val_loss: 0.3226 - val_accuracy: 0.9341 - 24s/epoch - 69ms/step
Epoch 56/100
352/352 - 24s - loss: 0.0211 - accuracy: 0.9933 - val_loss: 0.3647 - val_accuracy: 0.9357 - 24s/epoch - 67ms/step
Epoch 57/100
352/352 - 23s - loss: 0.0228 - accuracy: 0.9928 - val_loss: 0.3292 - val_accuracy: 0.9345 - 23s/epoch - 66ms/step
Epoch 58/100
352/352 - 24s - loss: 0.0195 - accuracy: 0.9938 - val_loss: 0.3539 - val_accuracy: 0.9339 - 24s/epoch - 69ms/step
Epoch 59/100
352/352 - 24s - loss: 0.0211 - accuracy: 0.9928 - val_loss: 0.3298 - val_accuracy: 0.9359 - 24s/epoch - 69ms/step
Epoch 60/100
352/352 - 24s - loss: 0.0185 - accuracy: 0.9940 - val_loss: 0.3734 - val_accuracy: 0.9337 - 24s/epoch - 69ms/step
Epoch 61/100
352/352 - 25s - loss: 0.0216 - accuracy: 0.9929 - val_loss: 0.3433 - val_accuracy: 0.9325 - 25s/epoch - 70ms/step
Epoch 62/100
352/352 - 24s - loss: 0.0201 - accuracy: 0.9934 - val_loss: 0.3436 - val_accuracy: 0.9343 - 24s/ep

...

Epoch 60/100
352/352 - 29s - loss: 2.1795 - accuracy: 0.2105 - val_loss: 2.2319 - val_accuracy: 0.1893 - 29s/epoch - 82ms/step
Epoch 61/100
352/352 - 30s - loss: 2.1573 - accuracy: 0.2209 - val_loss: 2.2182 - val_accuracy: 0.1899 - 30s/epoch - 85ms/step
Epoch 62/100
352/352 - 30s - loss: 2.1348 - accuracy: 0.2307 - val_loss: 2.2075 - val_accuracy: 0.1972 - 30s/epoch - 84ms/step
Epoch 63/100
352/352 - 29s - loss: 2.1094 - accuracy: 0.2400 - val_loss: 2.2004 - val_accuracy: 0.2049 - 29s/epoch - 82ms/step
Epoch 64/100
352/352 - 28s - loss: 2.0845 - accuracy: 0.2510 - val_loss: 2.1848 - val_accuracy: 0.2097 - 28s/epoch - 80ms/step
Epoch 65/100
352/352 - 28s - loss: 2.0613 - accuracy: 0.2590 - val_loss: 2.1830 - val_accuracy: 0.2137 - 28s/epoch - 79ms/step
Epoch 66/100
352/352 - 29s - loss: 2.0471 - accuracy: 0.2662 - val_loss: 2.1769 - val_accuracy: 0.2207 - 29s/epoch - 82ms/step
Epoch 67/100
352/352 - 29s - loss: 2.0352 - accuracy: 0.2715 - val_loss: 2.1721 - val_accuracy: 0.2198 - 29s/ep

Epoch 20/100
352/352 - 70s - loss: 2.1909 - accuracy: 0.1958 - val_loss: 2.3104 - val_accuracy: 0.1407 - 70s/epoch - 199ms/step
Epoch 21/100
352/352 - 71s - loss: 2.1880 - accuracy: 0.1957 - val_loss: 2.3191 - val_accuracy: 0.1391 - 71s/epoch - 200ms/step
Epoch 22/100
352/352 - 71s - loss: 2.1758 - accuracy: 0.2037 - val_loss: 2.3291 - val_accuracy: 0.1375 - 71s/epoch - 202ms/step
Epoch 23/100
352/352 - 70s - loss: 2.1772 - accuracy: 0.2012 - val_loss: 2.3161 - val_accuracy: 0.1420 - 70s/epoch - 200ms/step
Epoch 24/100
352/352 - 71s - loss: 2.1638 - accuracy: 0.2082 - val_loss: 2.3324 - val_accuracy: 0.1429 - 71s/epoch - 203ms/step
Epoch 25/100
352/352 - 70s - loss: 2.1577 - accuracy: 0.2122 - val_loss: 2.3261 - val_accuracy: 0.1363 - 70s/epoch - 199ms/step
Epoch 26/100
352/352 - 71s - loss: 2.1509 - accuracy: 0.2143 - val_loss: 2.3466 - val_accuracy: 0.1407 - 71s/epoch - 201ms/step
Epoch 27/100
352/352 - 71s - loss: 2.1425 - accuracy: 0.2182 - val_loss: 2.3659 - val_accuracy: 0.1398 -

352/352 - 72s - loss: 1.7718 - accuracy: 0.3621 - val_loss: 2.8717 - val_accuracy: 0.1255 - 72s/epoch - 203ms/step
Epoch 85/100
352/352 - 71s - loss: 1.7581 - accuracy: 0.3672 - val_loss: 2.9464 - val_accuracy: 0.1235 - 71s/epoch - 202ms/step
Epoch 86/100
352/352 - 71s - loss: 1.7535 - accuracy: 0.3641 - val_loss: 3.0243 - val_accuracy: 0.1235 - 71s/epoch - 203ms/step
Epoch 87/100
352/352 - 71s - loss: 1.7453 - accuracy: 0.3716 - val_loss: 3.0143 - val_accuracy: 0.1242 - 71s/epoch - 200ms/step
Epoch 88/100
352/352 - 72s - loss: 1.7409 - accuracy: 0.3718 - val_loss: 3.0274 - val_accuracy: 0.1253 - 72s/epoch - 204ms/step
Epoch 89/100
352/352 - 70s - loss: 1.7408 - accuracy: 0.3718 - val_loss: 3.0321 - val_accuracy: 0.1240 - 70s/epoch - 200ms/step
Epoch 90/100
352/352 - 71s - loss: 1.7359 - accuracy: 0.3755 - val_loss: 2.9729 - val_accuracy: 0.1248 - 71s/epoch - 203ms/step
Epoch 91/100
352/352 - 72s - loss: 1.9254 - accuracy: 0.3094 - val_loss: 2.7403 - val_accuracy: 0.1261 - 72s/epoch - 

Epoch 45/100
352/352 - 64s - loss: 2.0594 - accuracy: 0.2469 - val_loss: 2.5837 - val_accuracy: 0.1057 - 64s/epoch - 181ms/step
Epoch 46/100
352/352 - 62s - loss: 2.0496 - accuracy: 0.2508 - val_loss: 2.5880 - val_accuracy: 0.1071 - 62s/epoch - 177ms/step
Epoch 47/100
352/352 - 64s - loss: 2.0392 - accuracy: 0.2534 - val_loss: 2.5792 - val_accuracy: 0.1056 - 64s/epoch - 180ms/step
Epoch 48/100
352/352 - 62s - loss: 2.0365 - accuracy: 0.2558 - val_loss: 2.6033 - val_accuracy: 0.1058 - 62s/epoch - 177ms/step
Epoch 49/100
352/352 - 64s - loss: 2.0252 - accuracy: 0.2591 - val_loss: 2.6733 - val_accuracy: 0.1053 - 64s/epoch - 181ms/step
Epoch 50/100
352/352 - 64s - loss: 2.0336 - accuracy: 0.2574 - val_loss: 2.6036 - val_accuracy: 0.1047 - 64s/epoch - 180ms/step
Epoch 51/100
352/352 - 62s - loss: 2.0153 - accuracy: 0.2634 - val_loss: 2.6392 - val_accuracy: 0.1075 - 62s/epoch - 175ms/step
Epoch 52/100
352/352 - 47s - loss: 2.0034 - accuracy: 0.2696 - val_loss: 2.6483 - val_accuracy: 0.1069 -

                                                                 
 dense_19 (Dense)            (None, 10)                650       
                                                                 
Total params: 57482 (224.54 KB)
Trainable params: 57482 (224.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
from tensorflow.python.framework.convert_to_constants import  convert_variables_to_constants_v2_as_graph

def get_flops(model):
    concrete = tf.function(lambda inputs: model(inputs))
    concrete_func = concrete.get_concrete_function(
        [tf.TensorSpec([1, *inputs.shape[1:]]) for inputs in model.inputs])
    frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(concrete_func, lower_control_flow=False)
    with tf.Graph().as_default() as graph:
        tf.graph_util.import_graph_def(graph_def, name='')
        run_meta = tf.compat.v1.RunMetadata()
        opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        flops = tf.compat.v1.profiler.profile(graph=graph, run_meta=run_meta, cmd="op", options=opts)
        return flops.total_float_ops

print("The FLOPs is:{}".format(get_flops(model)) ,flush=True )

The FLOPs is:115516
